# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 17

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
df = pd.read_csv('../data/data_week' + str(gameweek) + '.csv', index_col=0, encoding='latin-1')
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,games played,adjusted points,adjusted points per game,form 5
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,24,100.0,100.0,0,69140,0,0,-3,3,0.8,0,2,1.3,0.3,0,Shkodran,0.8,2,0,7.5,False,19.8,90,NaN,2019-11-28T23:00:21.541666Z,52,0,0,0,69140.jpg,4.0,0,0,Mustafi,0.3,False,NaN,a,1,3,54.0,4,7831,116,32097,285,0.2,0.8,Mustafi,0,1.000000,4.281881,4.281881,NaN
2,0,0,26,0.0,75.0,0,98745,0,0,-1,1,22.7,0,2,0.0,-0.2,0,HÃÂ©ctor,0.2,6,0,6.9,False,30.8,262,Hamstring injury - Expected back 21 Dec,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,0.3,0,0,BellerÃÂ­n,0.3,False,NaN,i,1,3,15.0,1,34609,34,33648,1577,0.0,0.2,BellerÃÂ­n,2,3.333333,4.088660,1.226598,NaN
3,2,1,196,75.0,100.0,1,111457,0,0,-3,3,149.6,0,2,1.6,1.1,0,Sead,1.6,17,0,37.8,False,174.4,928,Ankle injury - 75% chance of playing,2019-12-15T19:30:19.136195Z,52,0,0,0,111457.jpg,1.9,0,0,Kolasinac,0.6,False,NaN,d,1,3,55.0,27,46926,335,109248,1839,0.3,5.2,Kolasinac,2,14.210526,37.079065,2.609268,2.506510
4,2,3,124,100.0,100.0,1,154043,0,0,-4,4,127.9,1,2,1.1,0.1,1,Ainsley,0.6,14,0,32.9,False,173.0,670,NaN,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.5,1,0,Maitland-Niles,2.6,False,NaN,a,1,3,33.0,20,546474,920,567069,2824,0.1,4.3,Maitland-Niles,0,8.000000,23.383351,2.922919,2.502208
5,0,2,224,0.0,NaN,2,39476,0,0,-1,1,30.7,0,2,0.0,-0.1,0,Sokratis,0.4,23,1,44.3,False,340.2,1350,Suspended until 26 Dec,2019-12-15T19:30:19.149412Z,49,0,0,0,39476.jpg,2.2,0,0,Papastathopoulos,1.4,False,NaN,s,1,3,71.0,33,107358,290,144034,6092,0.1,6.7,Sokratis,5,15.000000,33.948971,2.263265,1.515779


In [4]:
# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [5]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [7]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [8]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [9]:
metric = 'valuePoints metric'
#metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
411,PatrÃÂ­cio,Wolves,52,16.842105,64,72.493595,3.8,4.304307,3.709349,1.887564
341,Gazzaniga,Tottenham,46,10.967742,34,42.878194,3.1,3.909482,3.593710,1.822806
494,Ramsdale,Bournemouth,46,17.073171,70,65.678816,4.1,3.846902,4.224227,1.793627
131,Guaita,Crystal Palace,51,15.000000,69,59.537006,4.6,3.969134,6.570066,1.757562
47,Ryan,Brighton,48,17.000000,68,65.331051,4.0,3.843003,3.546943,1.754083
235,de Gea,Manchester Utd,55,17.096774,53,69.718133,3.1,4.077853,3.570257,1.738802
280,Krul,Norwich City,45,14.857143,52,54.541959,3.5,3.671093,2.254985,1.730570
471,Henderson,Sheffield Utd,48,16.000000,64,59.619621,4.0,3.726226,3.687495,1.700782
168,Schmeichel,Leicester City,54,17.073171,70,67.429636,4.1,3.949450,3.303809,1.699573


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
297,Lundstram,Sheffield Utd,52,17.115385,89,87.923184,5.2,5.137085,4.950149,2.252761
182,Alexander-Arnold,Liverpool,73,16.818182,74,89.175785,4.4,5.302344,4.104001,1.962486
208,Otamendi,Manchester City,50,11.000000,44,47.127475,4.0,4.284316,5.272050,1.916004
185,Matip,Liverpool,52,6.956522,32,29.916920,4.6,4.300557,NaN,1.885919
181,Robertson,Liverpool,71,16.976744,73,84.115817,4.3,4.954767,3.462681,1.859491
164,SÃÂ¶yÃÂ¼ncÃÂ¼,Leicester City,51,16.976744,73,71.234685,4.3,4.196016,5.073906,1.858027
162,Evans,Leicester City,52,17.111111,77,72.475379,4.5,4.235574,5.677803,1.857422
405,Boly,Wolves,48,8.095238,34,32.618246,4.2,4.029313,NaN,1.839121
161,Chilwell,Leicester City,58,13.877551,68,61.318072,4.9,4.418508,4.181216,1.834686


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
215,De Bruyne,Manchester City,103,16.111111,116,104.660027,7.2,6.496140,6.763897,2.024122
191,Salah,Liverpool,122,14.054054,104,95.990860,7.4,6.830119,9.488022,1.955457
196,Keita,Liverpool,58,3.018868,16,14.042873,5.3,4.651702,11.362441,1.931514
171,Maddison,Leicester City,78,16.000000,88,85.543644,5.5,5.346478,6.696922,1.914346
431,Pulisic,Chelsea,74,14.042553,66,70.037786,4.7,4.987539,4.318254,1.833456
214,Sterling,Manchester City,118,15.964912,91,98.553096,5.7,6.173106,6.458342,1.797061
192,ManÃÂ©,Liverpool,122,15.972222,115,100.124111,7.2,6.268640,7.081143,1.794706
29,Grealish,Aston Villa,62,15.000000,66,66.787419,4.4,4.452495,4.440244,1.788165
219,David Silva,Manchester City,75,14.905660,79,71.878101,5.3,4.822202,4.824013,1.760819


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
167,Iheanacho,Leicester City,57,3.013699,22,18.190566,7.3,6.035961,NaN,2.528186
278,Pukki,Norwich City,66,16.909091,93,88.735849,5.5,5.247819,6.665413,2.042711
233,Rashford,Manchester Utd,91,16.885246,103,103.626415,6.1,6.137098,6.930580,2.034428
460,Abraham,Chelsea,80,15.967742,99,90.535849,6.2,5.669922,4.783298,2.004620
166,Vardy,Leicester City,101,17.051282,133,107.526415,7.8,6.306060,8.106983,1.984255
502,Maupay,Brighton,58,17.073171,70,76.200000,4.1,4.463143,5.513333,1.853219
409,JimÃÂ©nez,Wolves,75,17.000000,85,85.826415,5.0,5.048613,4.991560,1.843493
313,Ings,Southampton,64,16.862745,86,76.845283,5.1,4.557104,4.879809,1.801354
68,King,Bournemouth,61,13.000000,52,54.535849,4.0,4.195065,3.073627,1.698532


Below we compile a list sorting players based on a given metric (irrespective of position).

In [10]:
#metric = 'adjusted points per game'
metric = 'form 5'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,valuePoints metric
id,,,,,,,,,,
196,Keita,Liverpool,58,3.018868,16,14.042873,5.3,4.651702,11.362441,1.931514
191,Salah,Liverpool,122,14.054054,104,95.990860,7.4,6.830119,9.488022,1.955457
166,Vardy,Leicester City,101,17.051282,133,107.526415,7.8,6.306060,8.106983,1.984255
192,ManÃÂ©,Liverpool,122,15.972222,115,100.124111,7.2,6.268640,7.081143,1.794706
233,Rashford,Manchester Utd,91,16.885246,103,103.626415,6.1,6.137098,6.930580,2.034428
215,De Bruyne,Manchester City,103,16.111111,116,104.660027,7.2,6.496140,6.763897,2.024122
171,Maddison,Leicester City,78,16.000000,88,85.543644,5.5,5.346478,6.696922,1.914346
278,Pukki,Norwich City,66,16.909091,93,88.735849,5.5,5.247819,6.665413,2.042711
131,Guaita,Crystal Palace,51,15.000000,69,59.537006,4.6,3.969134,6.570066,1.757562
